# MLP para la clasificación de productos de Mercado Libre


In [ ]:
!pip install PyDrive

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
mkdir -p data

In [ ]:
# Meli Data
downloaded = drive.CreateFile({'id':"1TpzCRwb4hLMkPwlv1jNy1clpBBg0ajvL"})   # replace the id with id of file you want to access
downloaded.GetContentFile('./data/meli-challenge-2019.tar.bz2')        # replace the file name with your file

# Diccionary
downloaded = drive.CreateFile({'id':"11_zHF4AjPRUjkW_x9CFKJGiPBm6lCKO2"})   # replace the id with id of file you want to access
downloaded.GetContentFile('./data/SBW-vectors-300-min5.txt.bz2')        # replace the file name with your file


In [ ]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2020 NVIDIA Corporation
Built on Wed_Jul_22_19:09:09_PDT_2020
Cuda compilation tools, release 11.0, V11.0.221
Build cuda_11.0_bu.TC445_37.28845127_0


In [ ]:
!pip install torch==1.7.1+cu110 -f https://download.pytorch.org/whl/torch_stable.html

Looking in links: https://download.pytorch.org/whl/torch_stable.html
     |███████████████████████         | 834.1MB 1.3MB/s eta 0:04:16tcmalloc: large alloc 1147494400 bytes == 0x5615e17de000 @  0x7ffa77fac615 0x5615a7bdc06c 0x5615a7cbbeba 0x5615a7bdee8d 0x5615a7cd099d 0x5615a7c52fe9 0x5615a7c4db0e 0x5615a7be077a 0x5615a7c52e50 0x5615a7c4db0e 0x5615a7be077a 0x5615a7c4f86a 0x5615a7cd17c6 0x5615a7c4eee2 0x5615a7cd17c6 0x5615a7c4eee2 0x5615a7cd17c6 0x5615a7c4eee2 0x5615a7cd17c6 0x5615a7d53431 0x5615a7cb4049 0x5615a7c1ec84 0x5615a7bdf8e9 0x5615a7c53ade 0x5615a7be069a 0x5615a7c4ea45 0x5615a7c4de0d 0x5615a7be077a 0x5615a7c4ea45 0x5615a7be069a 0x5615a7c4ea45
     |█████████████████████████████▏  | 1055.7MB 63.7MB/s eta 0:00:02tcmalloc: large alloc 1434370048 bytes == 0x561625e34000 @  0x7ffa77fac615 0x5615a7bdc06c 0x5615a7cbbeba 0x5615a7bdee8d 0x5615a7cd099d 0x5615a7c52fe9 0x5615a7c4db0e 0x5615a7be077a 0x5615a7c52e50 0x5615a7c4db0e 0x5615a7be077a 0x5615a7c4f86a 0x5615a7cd17c6 0x5615a7c4eee2 

In [ ]:
!pip install --upgrade gensim mlflow tqdm

     |████████████████████████████████| 24.2MB 137kB/s 
     |████████████████████████████████| 14.2MB 47.5MB/s 
     |████████████████████████████████| 81kB 9.8MB/s 
     |████████████████████████████████| 163kB 31.9MB/s 
     |████████████████████████████████| 61kB 9.3MB/s 
     |████████████████████████████████| 1.1MB 46.3MB/s 
     |████████████████████████████████| 81kB 10.6MB/s 
     |████████████████████████████████| 153kB 55.4MB/s 
     |████████████████████████████████| 71kB 10.6MB/s 
     |████████████████████████████████| 81kB 11.4MB/s 
     |████████████████████████████████| 61kB 9.6MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.14.3-cp37-none-any.whl size=100557 sha256=00d4c926d50fdeb691d2cd8797c8f19e1e8a55fe019925d244d0819e4a9e158c
  Stored in directory: /root/.cache/pip/wheels/5b/24/f3/34d8e3964dac4ba849d844273c49a679111b00d5799ebb934a
  Created wheel for alembic: filename=alembic-1.4.1-py2.py3-none-any.whl size=158155 sha256=90791db2e6858392ec662db8

In [ ]:
%%bash

#mkdir -p data
#curl -L https://cs.famaf.unc.edu.ar/\~ccardellino/resources/diplodatos/meli-challenge-2019.tar.bz2 -o ./data/meli-challenge-2019.tar.bz2
#curl -L http://cs.famaf.unc.edu.ar/~ccardellino/SBWCE/SBW-vectors-300-min5.txt.bz2 -o ./data/SBW-vectors-300-min5.txt.bz2
tar jxvf ./data/meli-challenge-2019.tar.bz2 -C ./data/
bunzip2 ./data/SBW-vectors-300-min5.txt.bz2

meli-challenge-2019/
meli-challenge-2019/spanish.train.csv.gz
meli-challenge-2019/portuguese.train.csv.gz
meli-challenge-2019/spanish.test.csv.gz
meli-challenge-2019/portuguese.test.csv.gz


In [ ]:
# para ahicar el dataset de train 
# Acordate de corregir en las funciones de prepocesamiento

%%bash

zcat ./data/meli-challenge-2019/spanish-train.csv.gz | head -n 1000000 | gzip > ./data/meli-challenge-2019/spanish-train-debug.csv.gz

gzip: ./data/meli-challenge-2019/spanish-train.csv.gz: No such file or directory


# MLflow y un experimento sencillo

## Librerías

In [ ]:
import gzip
import mlflow
import pandas as pd
import tempfile
import torch
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F

from gensim import corpora
from gensim.models import KeyedVectors
from gensim.parsing import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import balanced_accuracy_score
from torch.utils.data import IterableDataset, DataLoader
from tqdm.notebook import tqdm, trange

## Dataset

Nos basaremos en el dataset que creamos para el [notebook 2](./2_datasets.ipynb), con la diferencia de que pasaremos el dataframe de manera directa (esto es para poder hacer split en train/test).

In [ ]:
!zcat ./data/meli-challenge-2019/spanish.train.csv.gz | wc -l

6119101


In [ ]:
import numpy as np

train_index = np.random.rand(6119101) <= 0.8

with gzip.open("./data/meli-challenge-2019/spanish.train.csv.gz", "rt") as spanish_corpus,\
    gzip.open("./data/meli-challenge-2019/spanish-train.csv.gz", "wt") as spanish_train,\
    gzip.open("./data/meli-challenge-2019/spanish-validation.csv.gz", "wt") as spanish_validation:
  for idx, line in enumerate(tqdm(spanish_corpus, total=6119101)):
    if train_index[idx]:
      spanish_train.write(line)
    else:
      spanish_validation.write(line)

  0%|          | 0/6119101 [00:00<?, ?it/s]

In [ ]:
!zcat ./data/meli-challenge-2019/spanish-train.csv.gz | wc -l

4895870


In [ ]:
!zcat ./data/meli-challenge-2019/spanish-validation.csv.gz | wc -l

1223231


In [ ]:
!zcat ./data/meli-challenge-2019/spanish.test.csv.gz | wc -l

63681


In [ ]:
!zcat ./data/meli-challenge-2019/spanish-validation.csv.gz | head

spanish,reliable,Cómoda Toillete Escritorio Y Mesas De Luz Reina Ana Roble ,DRAWERS
spanish,reliable,Casio Fx 19 - Calculadora Cientifica,CALCULATORS
spanish,reliable,Mochila Las Oreiro Original!súper Oferta!envio Gratis!,BACKPACKS
spanish,reliable,Controlador Fiscal Homologado Elitronic Cr-50 *envío Gratis*,PRINTERS
spanish,reliable,Rifle Fox Pr900w Pcp Mira Cargador Doble Inflador Correa,AIRSOFT_GUNS
spanish,reliable,Rolling Stone 91 Rolling Stones,MAGAZINES
spanish,reliable,Armazones De Lectura Emporio Armani 100 % Original 9512,GLASSES_FRAMES
spanish,reliable,Memoria Pc 8gb Kingston Hyperx Fury Ddr3 1600mhz Azul,RAM_MEMORY_MODULES
spanish,reliable,Dimm 128mb 4 Chips 133,RAM_MEMORY_MODULES
spanish,reliable,"Bajo Performer By Spector, Muy Buen Estado!",BASS_GUITARS


In [ ]:
import csv
import random

class MeliChallengeDataset(IterableDataset):
    def __init__(self,
                 dataset_path,
                 dataset_size,
                 random_buffer_size=2048,
                 transform=None):
        assert random_buffer_size > 0
        self.dataset_path = dataset_path
        self.dataset_size = dataset_size
        self.random_buffer_size = random_buffer_size
        self.transform = transform
    
    def __len__(self):
        return self.dataset_size

    def __iter__(self):
        try:
            with gzip.open(self.dataset_path, "rt") as dataset:
                shuffle_buffer = []
                csv_reader = csv.reader(dataset)
                next(csv_reader)

                for line in csv_reader:
                    _, label_quality, title, category = line

                    if self.random_buffer_size == 1:
                        item = {
                            "title": f"{label_quality} {title}",
                            "category": category
                        }
                        if self.transform:
                            item = self.transform(item)
                        yield item
                    else:
                        shuffle_buffer.append({
                            "title": f"{label_quality} {title}",
                            "category": category
                        })

                        if len(shuffle_buffer) == self.random_buffer_size:
                            random.shuffle(shuffle_buffer)
                            for item in shuffle_buffer:
                                if self.transform:
                                    item = self.transform(item)
                                yield item
                            shuffle_buffer = []

            if len(shuffle_buffer) > 0:
                random.shuffle(shuffle_buffer)
                for item in shuffle_buffer:
                    if self.transform:
                        item = self.transform(item)
                    yield item
        except GeneratorExit:
            return

## Preprocesamiento

En este caso vamos a utilizar un sólo módulo para transformar los datos de MeLi. Este se encargará de preprocesar el texto (i.e. normalizarlo) y transformará las palabras en índices de un diccionario para luego poder pasar una secuencia de palabras para buscar en la matriz de embeddings y así permitir mayor manipulación de los embeddings (en lugar de utilizar embeddings fijos).

In [ ]:
class RawDataProcessor:
    def __init__(self, 
                 dataset_path,
                 dataset_size,
                 ignore_header=True, 
                 filters=None, 
                 vocab_size=50000):
        if filters:
            self.filters = filters
        else:
            self.filters = [
                lambda s: s.lower(),
                preprocessing.strip_tags,
                preprocessing.strip_punctuation,
                preprocessing.strip_multiple_whitespaces,
                preprocessing.strip_numeric,
                preprocessing.remove_stopwords,
                preprocessing.strip_short,
            ]
        
        # Create dictionary based on all the reviews (with corresponding preprocessing)
        self.dictionary = corpora.Dictionary()
        labels = set()

        with gzip.open(dataset_path, "rt") as dataset:
            csv_reader = csv.reader(dataset)
            if ignore_header:
              next(csv_reader)
            for line in tqdm(csv_reader, total=dataset_size):
                _, label_quality, title, category = line
                self.dictionary.add_documents(
                    [self._preprocess_string(f"{label_quality} {title}")]
                )
                labels.add(category)
        # Filter the dictionary and compactify it (make the indices continous)
        self.dictionary.filter_extremes(no_below=2, no_above=1, keep_n=vocab_size)
        self.dictionary.compactify()
        # Add a couple of special tokens
        self.dictionary.patch_with_special_tokens({
            "[PAD]": 0,
            "[UNK]": 1
        })
        self.idx_to_target = sorted(labels)
        self.target_to_idx = {t: i for i, t in enumerate(self.idx_to_target, start=1)}

    def _preprocess_string(self, string):
        return preprocessing.preprocess_string(string, filters=self.filters)

    def _sentence_to_indices(self, sentence):
        return self.dictionary.doc2idx(sentence, unknown_word_index=1)
    
    def encode_data(self, data):
        return self._sentence_to_indices(self._preprocess_string(data))
    
    def encode_target(self, target):
        return self.target_to_idx.get(target, 0)
    
    def __call__(self, item):
        if isinstance(item, list):
          data = [self.encode_data(i["title"]) for i in item]
          target = [self.encode_target(i["category"]) for i in item]
        else:
          data = self.encode_data(item["title"])
          target = self.encode_target(item["category"])
        
        return {
            "data": data,
            "target": target
        }

## Lectura de datos

En esta ocasión, leeremos los datos de MeLi y lo dividiremos en subconjuntos de entrenamiento y evaluación.

In [ ]:
preprocess = RawDataProcessor(
    "./data/meli-challenge-2019/spanish-train-debug.csv.gz",
    10000
#    preprocessing_buffer_size=100000
)

  0%|          | 0/10000 [00:00<?, ?it/s]

In [ ]:
train_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish-train-debug.csv.gz",
    10000,
    random_buffer_size=2048,
    transform=preprocess
)
validation_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish-validation.csv.gz",
    1224416,
    random_buffer_size=1,
    transform=preprocess
)

test_dataset = MeliChallengeDataset(
    "./data/meli-challenge-2019/spanish.test.csv.gz",
    63681,
    random_buffer_size=1,
    transform=preprocess
)

print(f"Datasets loaded with {len(train_dataset)} training elements and {len(validation_dataset)} validation elements")
print(f"Sample train element:\n{next(iter(train_dataset))}")

Datasets loaded with 10000 training elements and 1224416 validation elements
Sample train element:
{'data': [4, 436, 437, 1, 299], 'target': 342}


## Collation function

Como en este caso trabajamos con secuencias de palabras (representadas por sus índices en un vocabulario), cuando queremos buscar un *batch* de datos, el `DataLoader` de PyTorch espera que los datos del *batch* tengan la misma dimensión (para poder llevarlos todos a un tensor de dimensión fija). Esto lo podemos lograr mediante el parámetro de `collate_fn`. En particular, esta función se encarga de tomar varios elementos de un `Dataset` y combinarlos de manera que puedan ser devueltos como un tensor de PyTorch. Muchas veces la `collate_fn` que viene por defecto en `DataLoader` sirve (como se vio en el notbook 2), pero este no es el caso. Se define un módulo `PadSequences` que toma un valor mínimo, opcionalmente un valor máximo y un valor de relleno (*pad*) y dada una lista de secuencias, devuelve un tensor con *padding* sobre dichas secuencias.

In [ ]:
class PadSequences:
    def __init__(self, pad_value=0, max_length=None, min_length=1):
        assert max_length is None or min_length <= max_length
        self.pad_value = pad_value
        self.max_length = max_length
        self.min_length = min_length

    def __call__(self, items):
        data, target = list(zip(*[(item["data"], item["target"]) for item in items]))
        seq_lengths = [len(d) for d in data]

        if self.max_length:
            max_length = self.max_length
            seq_lengths = [min(self.max_length, l) for l in seq_lengths]
        else:
            max_length = max(self.min_length, max(seq_lengths))

        data = [d[:l] + [self.pad_value] * (max_length - l)
                for d, l in zip(data, seq_lengths)]
            
        return {
            "data": torch.LongTensor(data),
            "target": torch.LongTensor(target)
        }

## DataLoaders

Ya habiendo definido nuestros conjuntos de datos y nuestra `collation_fn`, podemos definir nuestros `DataLoader`s, uno para entrenamiento y otro para evaluación. Ver que la diferencia fundamental está en `shuffle`, no queremos mezclar los valores de evaluación cada vez que evaluamos porque al evaluar mediante *mini-batchs* nos puede generar inconsistencias.

In [ ]:
# Batch size modificado. Original 128

pad_sequences = PadSequences()
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=False,
                          collate_fn=pad_sequences, drop_last=False)
validation_loader = DataLoader(validation_dataset, batch_size=64, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False,
                               collate_fn=pad_sequences, drop_last=False)

In [ ]:
next(iter(train_loader))

{'data': tensor([[   4, 2418,    1, 2417,    1, 2418,    1, 2417,  962,    0],
         [   4,  236,  237,  235,    1,  234,  232,  231,  233,    0],
         [   4, 1186, 2095, 1517,    0,    0,    0,    0,    0,    0],
         [   4, 1698,    1, 1388,    1, 2450,    1, 1126,  892,    0],
         [   4,  447,  672, 2836,  360,    0,    0,    0,    0,    0],
         [   4,  767,  759,  299,  768,   97,    0,    0,    0,    0],
         [   4,  569, 2094,  327, 2092, 2093, 1287,    0,    0,    0],
         [   4, 1158,    1, 2072,  354,  283,    0,    0,    0,    0],
         [   4,    1,  498,  781,  353,  299,    0,    0,    0,    0],
         [   4, 1737,  507, 2942, 2946,  638, 1906,    0,    0,    0],
         [   4,  168,  165,  525,  846,  166,   99,  164,  937,    0],
         [   4,  336,    1,  762,    1,  777,    1,  381,  776,    0],
         [   4,  176,  833,  447,  832,  831, 1517, 2570,    0,    0],
         [   4,  118, 1244, 1277, 1742,  120, 1743,    0,    0,    0]

In [ ]:
next(iter(validation_loader))

{'data': tensor([[   4,  634, 1773, 3338,    0,    0,    0,    0,    0,    0],
         [   4,  145, 2132,    1,    9, 2494,  213,  365,  281,    0],
         [   4,  772,    1,    1,    1,  279,  281,    0,    0,    0],
         [   4, 1103, 1867,    1, 1548, 2634,  786,  245, 4752, 1407],
         [   4, 1521, 1522, 1521,    1,    0,    0,    0,    0,    0],
         [   4,  602, 4551,    1, 5222,    9,    0,    0,    0,    0],
         [   4,  613,  612, 2008,  857,  611,  557,  432,    0,    0],
         [   4,  556,    1,    0,    0,    0,    0,    0,    0,    0],
         [   4, 1016,    1,    1, 1474, 2360,  430,    0,    0,    0],
         [   4,  585, 2564, 2563, 2616,    1,    0,    0,    0,    0],
         [   4, 1348, 2058,  541, 1739, 2267,  142,    0,    0,    0],
         [   4,  864, 1355,  514, 1110, 1019,    1,    1,    0,    0],
         [   4,    1, 1270, 4937,    1,    1,    1,    0,    0,    0],
         [   4, 5632, 1214, 1419, 2264, 1002, 2528,    0,    0,    0]

## El modelo de clasificación

Para clasificación utilizaremos un perceptrón multicapa de dos capas ocultas. Claramente este modelo es naive y prácticamente todo lo que está *hardcodeado* (e.g. los tamaños de las capas o la cantidad de capas) podría ser parte de los parámetros del modelo. En particular, tenemos la capa de `Embeddings` que es rellenada con los valores de embeddings preentrenados.

In [ ]:
!head ./data/SBW-vectors-300-min5.txt 

1000653 300
de -0.029648 0.011336 0.019949 -0.088832 -0.025225 0.056844 0.025473 0.014068 0.163694 -0.067154 0.014738 0.027134 0.066443 -0.044846 -0.044987 -0.040898 0.030311 0.034196 -0.049240 0.008537 -0.068091 -0.087938 0.035300 0.149385 -0.012350 0.012613 0.029350 0.069596 0.039111 0.057652 0.069954 -0.066217 -0.041784 0.028623 0.026772 -0.066392 0.002953 -0.012188 -0.030363 0.040222 0.034858 0.027469 -0.029034 -0.048748 -0.038582 -0.051553 -0.033501 -0.019008 0.003043 0.110712 -0.025096 0.111082 0.035244 0.114207 0.010195 0.051511 -0.040649 -0.113944 0.044873 0.052011 0.067360 0.049054 -0.127085 -0.031846 0.032848 0.040825 -0.084873 0.059801 -0.067424 0.016531 -0.084565 0.057024 0.083288 -0.010136 -0.048508 0.051757 0.046664 0.018102 -0.052320 -0.000765 0.053662 -0.009967 0.082858 0.009068 0.054575 -0.003466 -0.023376 0.023069 0.088513 0.018504 -0.039503 -0.032980 -0.002139 0.000010 -0.107627 0.007699 0.046351 -0.003062 0.030500 0.113650 0.032536 -0.097301 -0.013734 0.098345 0.080

# Multi-layer perceptron

In [ ]:
class MeliChallengeClassifier(nn.Module):
    def __init__(self, 
                 pretrained_embeddings_path, 
                 dictionary,
                 n_labels,
                 vector_size,
                 freeze_embedings):
        super().__init__()
        embeddings_matrix = torch.randn(len(dictionary), vector_size)
        embeddings_matrix[0] = torch.zeros(vector_size)
        with open(pretrained_embeddings_path, "rt") as fh:
            next(fh)
            for line in fh:
                word, vector = line.strip().split(None, 1)
                if word in dictionary.token2id:
                    embeddings_matrix[dictionary.token2id[word]] =\
                        torch.FloatTensor([float(n) for n in vector.split()])
        self.embeddings = nn.Embedding.from_pretrained(embeddings_matrix,
                                                       freeze=freeze_embedings,
                                                       padding_idx=0)
        self.hidden1 = nn.Linear(vector_size, 128)
        self.hidden2 = nn.Linear(128, 128)
        self.output = nn.Linear(128, n_labels)
        self.vector_size = vector_size
    
    def forward(self, x):
        x = self.embeddings(x)
        x = torch.mean(x, dim=1)
        x = F.relu(self.hidden1(x))
        x = F.relu(self.hidden2(x))
        x = self.output(x)
        return x

In [ ]:
hidden_layers = [256, 128, 64, 32]
list(zip(hidden_layers[:-1], hidden_layers[1:]))

[(256, 128), (128, 64), (64, 32)]

In [ ]:
model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                preprocess.dictionary, 
                                len(preprocess.target_to_idx) + 1,
                                300, True)

In [ ]:
print(model)

MeliChallengeClassifier(
  (embeddings): Embedding(5634, 300, padding_idx=0)
  (hidden1): Linear(in_features=300, out_features=128, bias=True)
  (hidden2): Linear(in_features=128, out_features=128, bias=True)
  (output): Linear(in_features=128, out_features=605, bias=True)
)


## Experimento de MLflow

Por último, ya tenemos todos los bloques para construir nuestro experimento de MLflow. Anotamos un par de parámetros  (estos pueden ser todos los que se consideren necesarios) y lanzamos a correr nuestro experimento. Cada vez que finaliza un epoch guardamos algunas métricas. Al finalizar todos los epochs corremos algunas métricas extras de evaluación y guardamos algunos datos extra que nos servirán para calcular otras métricas a futuro.

In [ ]:
EPOCHS = 150

mlflow.set_experiment("multilayer perceptron first attempt")

with mlflow.start_run():
    mlflow.log_param("model_name", "mlp")
    mlflow.log_param("freeze_embedding", True)
    mlflow.log_params({
        "embedding_size": 50,
        "hidden1_size": 128,
        "hidden2_size": 128
    })
    model = MeliChallengeClassifier("./data/SBW-vectors-300-min5.txt", 
                                    preprocess.dictionary, 
                                    len(preprocess.target_to_idx) + 1,
                                    300, True)
    device = torch.device("cuda")
    model = model.to(device)
    loss = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
    for epoch in trange(EPOCHS):
        model.train()
        running_loss = []
        for idx, batch in enumerate(tqdm(train_loader)):
            optimizer.zero_grad()
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            loss_value = loss(output, target)
            loss_value.backward()
            optimizer.step()
            running_loss.append(loss_value.item())        
        mlflow.log_metric("train_loss", sum(running_loss) / len(running_loss), epoch)
        
        # Evaluación con los datos de validacion
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        with torch.no_grad():
          for batch in tqdm(validation_loader):
            data = batch["data"].to(device)
            target = batch["target"].to(device)
            output = model(data)
            running_loss.append(
                loss(output, target).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
        mlflow.log_metric("validation_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("validation_bacc", balanced_accuracy_score(targets, predictions), epoch)

        # Evaluación con los datos de test
        model.eval()
        running_loss = []
        targets = []
        predictions = []
        with torch.no_grad():
          for batch in tqdm(test_loader):
            data = batch["data"].to(device)
            target = batch["target"].to(device)
            output = model(data)
            running_loss.append(
                loss(output, target).item()
            )
            targets.extend(batch["target"].numpy())
            predictions.extend(output.argmax(axis=1).detach().cpu().numpy())
        mlflow.log_metric("test_loss", sum(running_loss) / len(running_loss), epoch)
        mlflow.log_metric("test_bacc", balanced_accuracy_score(targets, predictions), epoch)


    
    with tempfile.TemporaryDirectory() as tmpdirname:
        targets = []
        predictions = []
        for batch in tqdm(test_loader):
            data = batch["data"].to(device)
            target = batch["target"].type(torch.LongTensor).to(device)
            output = model(data)
            targets.extend(target.cpu().detach().numpy())
            predictions.extend(output.squeeze().cpu().detach().numpy())
        pd.DataFrame({"prediction": predictions, "target": targets}).to_csv(
            f"{tmpdirname}/predictions.csv.gz", index=False
        )
        mlflow.log_artifact(f"{tmpdirname}/predictions.csv.gz")

INFO: 'multilayer perceptron first attempt' does not exist. Creating a new experiment


RuntimeError: ignored

A pesar del error actual el código funciona. Se agotó el tiempo en Colab para correr con GPU. 

In [ ]:
!tar zcvf mlruns.tar.gz mlruns

In [ ]:
%%bash

# Una vez que lo descargan, lo descomprimen en su máquina con el siguiente comando
tar xvf mlruns.tar.gz

# Luego ejecutan en el lugar que lo descomprimieron lo siguiente
mlflow ui

Process is interrupted.
